# Import Required Libraries
Import the necessary libraries, including pathlib, datetime, and the custom modules.

In [1]:
# Import the necessary libraries
import pandas as pd
from pathlib import Path
import re
import sys
sys.path.append('src')

from datetime import datetime, timedelta

# Import custom modules
from data.loader import BankDataLoader
import config
from analysis.reports import *

# Load and Filter Data
Load the bank statements data using BankDataLoader and filter out transactions before a specified date.

In [2]:
# Load the bank statements data using BankDataLoader
loader = BankDataLoader(config.DATA_DIRS)
data = loader.load_statements()

# Filter out transactions before a specified date
filter_date = '2023-02-01'
data = data[data['Date'] >= filter_date]

# Dump the intermediate data to a CSV file
loader.dump_intermediate(data, 'intermediate.csv')

# Display the first few rows of the filtered data
data.head()

,Date,Description,Amount,Income,Expense,Category,Category1,Category2,Category3,Category12,Category123,Account,Tag
2544,2023-02-01,Gallipott,-78.90,0.0,-78.90,Expense:Misc:hair cuts,Expense,Misc,hair cuts,Expense:Misc,Expense:Misc:hair cuts,Discover,NaN
1073,2023-02-01,American Family Insurance,-257.05,0.0,-257.05,Expense:Insurance,Expense,Insurance,Unassigned,Expense:Insurance,Expense:Insurance:Unassigned,First Tech,NaN
2545,2023-02-01,HILTON ADVPURCH800236711 MEMPHIS TN,-136.16,0.0,-136.16,Expense:Unassigned:Unassigned,Expense,Unassigned,Unassigned,Expense:Unassigned,Expense:Unassigned:Unassigned,Discover,None
2546,2023-02-02,Amazon,-6.83,0.0,-6.83,Expense:Misc,Expense,Misc,Unassigned,Expense:Misc,Expense:Misc:Unassigned,Discover,NaN
2547,2023-02-02,FCHS Tower Theater,-56.95,0.0,-56.95,Expense:Entertainment,Expense,Entertainment,Unassigned,Expense:Entertainment,Expense:Entertainment:Unassigned,Discover,NaN


# Filter Data

In [3]:

this_exclude_tags = ['#exclude_investment', '#ccpayment', '#college529', '#IBondRedemption', 
                     '#exclude_car', '#exclude_Janet529', '#exclude_theft', '#wyd_travel',
                     'exclude_loan', '#exclude_repayment']

data = exclude_transactions(data, this_exclude_tags)

# Dump Intermediate Data
Dump the filtered data to an intermediate CSV file.

In [4]:
# Dump the intermediate data to a CSV file
loader.dump_intermediate(data, 'intermediate.csv')

unassigned = data[(data['Category12'] == 'Income:Unassigned') | (data['Category12'] == 'Expense:Unassigned')].sort_values(by='Amount', ascending=True)
#unassigned = unassigned[unassigned['Date'] >= '2024-11-01']
unassigned = unassigned[['Date', 'Description', 'Amount', 'Income', 'Category12', 'Tag']]
loader.dump_intermediate(unassigned, 'unassigned.csv')


# Generate Monthly Totals Report
Generate and display the monthly totals report using the monthly_totals function.

In [5]:
# Generate Monthly Totals Report
monthly_totals(data)

Total for 2023-02:  -1,486.56 (Income:  10,072.87, Expense: -11,559.43)
Total for 2023-03:   3,656.99 (Income:  15,155.19, Expense: -11,498.20)
Total for 2023-04:  -2,330.90 (Income:   8,811.47, Expense: -11,142.37)
Total for 2023-05:      80.75 (Income:  11,020.18, Expense: -10,939.43)
Total for 2023-06:    -107.66 (Income:   8,758.51, Expense:  -8,866.17)
Total for 2023-07:    -388.69 (Income:  10,823.20, Expense: -11,211.89)
Total for 2023-08:  -2,007.63 (Income:   9,127.89, Expense: -11,135.52)
Total for 2023-09:   1,362.30 (Income:  12,123.74, Expense: -10,761.44)
Total for 2023-10:  -1,173.63 (Income:   9,091.37, Expense: -10,265.00)
Total for 2023-11:  -1,740.64 (Income:   8,872.67, Expense: -10,613.31)
Total for 2023-12:   9,463.89 (Income:  20,624.44, Expense: -11,160.55)
Total for 2024-01:    -960.92 (Income:   9,298.39, Expense: -10,259.31)
Total for 2024-02:      34.31 (Income:   9,464.60, Expense:  -9,430.29)
Total for 2024-03:  -1,737.65 (Income:   9,465.00, Expense: -11,

In [6]:
# Pivot?
df = data

df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.to_period('M')
df['Amount'] = df['Amount'].fillna(0)

# Group by month
# df['Amount']

df_group = df.groupby([pd.Grouper(key='Month'), 'Category12'])['Amount'].sum().reset_index()
# Ensure every month/category12 pair has a valid value, which is 0 if NaN
df_group = df_group.pivot(index='Month', columns='Category12', values='Amount').fillna(0).reset_index()
df_group = df_group.melt(id_vars=['Month'], var_name='Category12', value_name='Amount')

# with pd.option_context('display.max_rows', None):
#     print(df_group) 
# df_group

# # Calculate the rolling average
df_group['Rolling_Avg'] = df_group.groupby('Category12')['Amount'].transform(lambda x: x.rolling(window=12, min_periods=1).mean())

# # Pivot to create a column for each category
df_group = df_group.pivot(index='Category12', columns='Month', values='Rolling_Avg').reset_index()
df_group = df_group.round(2)

# Reorder the Category12 column by the value of 2024-12
#df_group = df_group.sort_values(by='2024-12', ascending=False).reset_index(drop=True)

# Filter columns to show only months after 2024-01
df_group = df_group[['Category12'] + [col for col in df_group.columns if not str(col).startswith('Category') and str(col) >= '2024-01']]
print('Rolling 12 month average')
df_group


Rolling 12 month average


Month,Category12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01
0,Expense:Auto,-800.00,-732.65,-674.65,-666.95,-659.50,-728.73,-654.53,-696.13,-612.71,-609.22,-600.56,-611.76,-599.62
1,Expense:Charity,-417.30,-375.52,-371.34,-370.49,-377.14,-382.54,-392.95,-402.95,-404.19,-407.02,-408.93,-406.11,-446.24
2,Expense:Chiropractic,-113.74,-91.87,-113.74,-89.74,-135.37,-180.27,-225.53,-284.99,-363.65,-376.00,-387.25,-409.12,-387.25
3,Expense:Clothes,-130.81,-127.87,-152.70,-185.31,-189.20,-188.23,-177.25,-169.49,-175.37,-175.37,-160.23,-126.87,-140.19
4,Expense:College,0.00,0.00,0.00,0.00,0.00,-68.24,-68.24,-68.24,-68.24,-118.24,-175.17,-175.17,-175.17
5,Expense:Credit card payment,-1181.70,-1174.71,-1146.89,-1164.25,-1165.95,-1135.63,-1109.91,-1077.77,-1058.37,-989.12,-988.44,-968.54,-1007.30
6,Expense:Dining,-581.15,-634.35,-699.08,-744.60,-783.92,-878.86,-940.61,-1010.23,-1071.79,-1127.02,-1148.58,-1196.27,-1196.61
7,Expense:Education,-885.28,-869.92,-948.94,-968.86,-1002.27,-1002.58,-1023.25,-1045.00,-1072.07,-1114.66,-1158.47,-1160.68,-1188.38
8,Expense:Entertainment,-245.96,-216.48,-227.25,-222.84,-233.66,-225.05,-211.16,-207.64,-217.26,-234.53,-237.63,-246.01,-265.55
9,Expense:Gifts,-99.97,-99.97,-114.12,-97.26,-97.31,-97.31,-97.31,-97.31,-129.34,-125.30,-198.64,-167.53,-171.42


In [7]:
df = rolling_12M_avg_category(data, 'Category1')
df.loc['Total'] = df.sum(numeric_only=True)
print('Rolling 12 month average')
df

Rolling 12 month average


Month,Category1,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01
0,Expense,-10817.03,-10647.05,-10626.06,-10557.87,-10586.20,-10885.46,-11000.12,-11199.61,-11390.88,-11564.79,-11701.76,-11806.80,-11998.60
1,Income,11423.62,11380.39,10909.84,11188.86,11065.27,11134.67,11030.56,11042.88,11005.62,10993.42,11100.52,10814.53,10609.46
Total,NaN,606.59,733.34,283.78,630.99,479.07,249.21,30.44,-156.73,-385.26,-571.37,-601.24,-992.27,-1389.14


In [8]:
df = data
pivot_df = df.pivot_table(index='Category12', columns='Month', values='Amount', aggfunc='sum').fillna(0)
pivot_df = pivot_df.loc[:, pivot_df.columns >= '2024']
print('Actual numbers')
pivot_df.loc['Total'] = pivot_df.sum(numeric_only=True)
pivot_df

Actual numbers


Month,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01
Category12,,,,,,,,,,,,,
Expense:Auto,-501.76,-365.86,-396.26,-378.84,-552.85,-1116.46,-912.59,-941.65,-732.18,-480.35,-386.94,-575.37,-356.14
Expense:Charity,-313.56,-994.15,-298.18,-308.18,-388.18,-398.13,-448.13,-408.13,-343.11,-322.13,-361.12,-290.32,-795.13
Expense:Chiropractic,-262.50,0.00,-262.50,0.00,-547.54,-538.86,-805.63,-713.46,-943.96,-437.50,-135.00,-262.50,0.00
Expense:Clothes,-54.03,-33.59,-297.85,-391.34,-319.63,0.00,-116.65,-157.16,-70.50,0.00,0.00,-81.69,-213.86
Expense:College,0.00,0.00,0.00,0.00,0.00,-818.90,0.00,0.00,0.00,-600.00,-683.13,0.00,0.00
Expense:Credit card payment,-1202.19,-1008.28,-768.10,-1129.37,-856.68,-624.05,-1323.33,-868.45,-1238.21,-922.82,-835.96,-845.09,-1667.23
Expense:Dining,-822.16,-847.10,-951.19,-1099.35,-1307.55,-1726.22,-1279.99,-1411.67,-1223.53,-1281.98,-968.59,-1435.86,-826.27
Expense:Education,-1100.12,-1072.45,-1811.45,-1143.13,-1197.40,-90.00,-248.03,-1282.99,-1345.45,-1696.03,-1706.17,-1235.00,-1432.40
Expense:Entertainment,-897.28,-65.67,-175.91,-185.11,-228.81,-169.47,-200.44,-236.99,-131.54,-276.72,-75.62,-308.61,-1131.68
